# Converter o GRIB em um Ctl

In [1]:
import pygrib 
import numpy as np
from datetime import datetime
start = datetime.now()

# GRIB_PATH = r'GPOSNMC20170906122017090612P.inz.TQ0666L064.grb'
GRIB_PATH = r'GPOSNMC20170202122017020312P.grib2'
grib = pygrib.open(GRIB_PATH)
print(datetime.now() - start)
print("0:02:33.892469")

0:00:02.081724
0:02:33.892469


In [2]:
start = datetime.now()
cube_list = []

def has_levels(grib_message):
    if grib_message['typeOfLevel'] != 'isobaricInhPa':
        return False
    else:
        return True

def make_cube(grib_message):
    if has_levels(grib_message): 
        '''
            If the cube_list is empty or if the last element is different
            from the grib_message(param), only them it's goind to append to the list
        '''
        if not cube_list or cube_list[-1]['shortName'] != grib_message['shortName']:
            selected_vars = grib.select(name=grib_message['name'], typeOfLevel=grib_message['typeOfLevel'])
            n_levels = len(selected_vars)
            levels = np.array([grb['level'] for grb in selected_vars])
            indexes = np.argsort(levels)[::-1]

            cube_dict = {'name':grib_message['name'],
                        'units':grib_message['units'],
                        'levels':levels, 
                        'shortName':grib_message['shortName']}

            cube_list.append(cube_dict)	
        # The message can be in the surface or at a specific height, so if the message has no levels,
        # it's going to append a 2-Dimensional cube
    else: 
        level = grib_message['level']
        cube_dict = {'name':grib_message['name'],
                    'units':grib_message['units'],
                    'levels':level, 
                    'shortName':grib_message['shortName']}
        cube_list.append(cube_dict)

print(datetime.now() - start)

0:00:00.000502


In [3]:
start = datetime.now()
for msg in grib:
    make_cube(msg)

# thefile = open('test.txt', 'wa')

def write_title(path):
    thefile = open('test.txt', 'a')
    thefile.write('''dset ^%s\n* \nindex %s\n*\ntitle PRESSURE HISTORY    PTEC AGCM REVIS 1.0 2000  T066664   COLD\n*
dtype grib   255\n*\noptions yrev\n*\n'''%(GRIB_PATH, GRIB_PATH[:-5] + "idx"))
    thefile.close()
    
def write_xyt(msg_list):
    thefile = open('test.txt', 'a')
    msg = grib[10]
    first_lat = msg['latitudeOfFirstGridPointInDegrees']
    first_lon = msg['longitudeOfLastGridPoint']
    thefile.write('''xdef  %s linear    %s   0.1800000000\nydef  %s linear  %s   0.1800000000\ntdef     1 linear 12Z06SEP2017 6hr\n*\n''' % (len(msg['distinctLongitudes']), first_lon, len(msg['distinctLatitudes']), first_lat))
    thefile.close()
    
    
def write_msgs(msg_list):
    thefile = open('test.txt', 'a')
    for msg in cube_list:
        thefile.write('%s    %s    %s\n' % (msg['shortName'], 
                                      msg['name'],
                                      msg['units']))
    thefile.close()

def write_to_file(msg_list):
    for msg in msg_list:
        pass   
            
        
write_title(GRIB_PATH)
write_xyt(cube_list)
write_msgs(cube_list)
# write_to_file(cube_list)

In [4]:
msg = grib[2]
for k in msg.keys(): print(k)
print('---------------------------------------------------------------')
print(msg['parameterName'])

parametersVersion
UseEcmfConventions
GRIBEX_boustrophedonic
hundred
globalDomain
GRIBEditionNumber
tablesVersionLatest
grib2divider
angularPrecision
missingValue
ieeeFloats
isHindcast
section0Length
identifier
discipline
editionNumber
totalLength
sectionNumber
section1Length
numberOfSection
centre
centreDescription
subCentre
tablesVersion
masterDir
localTablesVersion
significanceOfReferenceTime
year
month
day
hour
minute
second
dataDate
julianDay
dataTime
productionStatusOfProcessedData
typeOfProcessedData
selectStepTemplateInterval
selectStepTemplateInstant
stepType
setCalendarId
deleteCalendarId
is_uerra
sectionNumber
grib2LocalSectionPresent
sectionNumber
gridDescriptionSectionPresent
section3Length
numberOfSection
sourceOfGridDefinition
numberOfDataPoints
numberOfOctectsForNumberOfPoints
interpretationOfNumberOfPoints
PLPresent
gridDefinitionTemplateNumber
gridDefinitionDescription
shapeOfTheEarth
scaleFactorOfRadiusOfSphericalEarth
scaledValueOfRadiusOfSphericalEarth
scaleFactorOf